In [1]:
import os
import ray
from datetime import datetime
from ray.tune import tune
from definitions import ROOT_DIR
from src.envs.environment_factory import EnvironmentFactory
from src.helpers.experiment_config import ExperimentConfig
from src.models.model_factory import ModelFactory
from ray.tune.logger import UnifiedLogger


task_name = "baodingBalls"
file_name = "sac_baodingBalls_P1.json"

config_path = os.path.join(
    ROOT_DIR,
    "configs",
    task_name,
    file_name
)
config = ExperimentConfig(config_path)
# env = EnvironmentFactory.register(config.env_name, **config.env_config)
env = EnvironmentFactory.register(config.env_name)
ModelFactory.register_models_from_config(config.policy_configs)


RS:> Registering Myo Envs


2022-09-02 19:51:49,653	WARNING deprecation.py:48 -- DeprecationWarning: `ray.rllib.agents.a3c` has been deprecated. Use `ray.rllib.algorithms.[a3c|a2c]` instead. This will raise an error in the future!
2022-09-02 19:51:49,685	WARNING deprecation.py:48 -- DeprecationWarning: `ray.rllib.agents.ddpg` has been deprecated. Use `ray.rllib.algorithms.ddpg` instead. This will raise an error in the future!
2022-09-02 19:51:49,691	WARNING deprecation.py:48 -- DeprecationWarning: `ray.rllib.agents.dqn` has been deprecated. Use `ray.rllib.algorithms.[dqn|simple_q|r2d2|apex_dqn]` instead. This will raise an error in the future!
2022-09-02 19:51:49,702	WARNING deprecation.py:48 -- DeprecationWarning: `ray.rllib.agents.impala` has been deprecated. Use `ray.rllib.algorithms.impala` instead. This will raise an error in the future!
2022-09-02 19:51:49,730	WARNING deprecation.py:48 -- DeprecationWarning: `ray.rllib.agents.sac` has been deprecated. Use `ray.rllib.algorithms.sac` instead. This will raise 

In [2]:
from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
from stable_baselines3.common.monitor import Monitor

def make_parallel_envs(env_name,env_config, num_env, tensorboard_log, start_index=0):

    def make_env(rank):
        def _thunk():
            env = EnvironmentFactory.register(env_name, **env_config)
            env = Monitor(env,tensorboard_log)
            return env

        return _thunk

    return SubprocVecEnv([make_env(i + start_index) for i in range(num_env)])

envs = make_parallel_envs(config.env_name, config.env_config,num_env=16,tensorboard_log="./static_goal_ppo/")

RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs


You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal co

RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs
RS:> Registering Myo Envs


You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal component by another module). Please make sure you are using pickle5 >= 0.0.10 because previous versions may leak memory.
You are using the 'pickle5' module, but the exact version is unknown (possibly carried as an internal co

In [3]:
from stable_baselines3 import PPO

policy_kwargs = dict(net_arch=[256,256])

model = PPO(
    "MlpPolicy",
    envs,
    policy_kwargs=policy_kwargs,
    verbose=2,a
    tensorboard_log="./original_ppo/",
    batch_size=1024,
    n_steps=1280,
    gamma=0.99,
    gae_lambda=0.95,
    n_epochs=6,
)
model.learn(total_timesteps=10000000)
model.save("original_ppo")

Using cpu device
Logging to ./original_ppo/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.3     |
|    ep_rew_mean     | -12.2    |
| time/              |          |
|    fps             | 1234     |
|    iterations      | 1        |
|    time_elapsed    | 16       |
|    total_timesteps | 20480    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 17.5        |
|    ep_rew_mean          | -11         |
| time/                   |             |
|    fps                  | 1198        |
|    iterations           | 2           |
|    time_elapsed         | 34          |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.015533159 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | -55.3       |
|    explained_variance

In [ ]:
# Enjoy trained agent
obs = env.reset()
for i in range(10000):
    env.sim.render(mode='window')
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    if dones:
      env.reset()


Creating window glfw
Pressed ESC
Quitting.


: 

: 

In [8]:
dones

True